# World Wide Products Inc.

Dalton Hahn (2762306)

## Product Data

https://www.kaggle.com/felixzhao/productdemandforecasting

## Goal

* Perform feature engineering on the dataset in order to enable time-series forecasting
* Use time-series forecasting to predict demand of a given product

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from statistics import mean, stdev
import string
from sklearn import preprocessing

In [3]:
df = pd.read_csv("../data/external/Historical Product Demand.csv")
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
